# Profiling of Demo
    Alan Morningstar and Steven Li
    Princeton University APC 524 Final Project
    December 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from src.dof import DiscreteDOF
from src.unitcell import HoneycombUnitCell
from src.lattice import Lattice
from src.gate import HoneycombGate

### Encapsulate demo code in a class

In [2]:
class Demo:
    """Demoing the apclattice project."""
    
    def __init__(self, L):
        """Initialize demo.
        
        :param L: List of two integers. Number of unit cells in each dimension of the honeycomb lattice.
        """
        # honeycomb unit cell, two sites per cell, 0 or 1 particles per site
        spc = 2
        max_charge = 1
        dof = spc * [DiscreteDOF(0, max_charge)]
        self.uc = HoneycombUnitCell(dof)
        # for now leave the system without particles and create the lattice
        nsites = np.prod(L)*spc
        vals = np.zeros(nsites, dtype=int)
        self.lat = Lattice(self.uc, L, vals, periodic=True)
        # gate to enact local dynamics
        self.gate = HoneycombGate(self.uc)
        
    def plot(self, s=10):
        """Make an image of the lattice and values of degrees of freedom on each site.
        
        :param s: Size of marker passed to plt.scatter.
        """
        # xy coordinates of each site
        x, y = self.lat.positions.T
        # color tan if 0, teal if 1
        cmap = np.asarray(['#E1BE6A', '#40B0A6'])
        # plot the positions
        fig, ax = plt.subplots()
        ax.scatter(x, y, marker='o', c = cmap[self.lat.vals], s=s)
        # labels
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_title('Honeycomb Lattice')
        
    def charge_blob(self):
        """Put a blob of charge in the middle of the lattice."""
        # center of the lattice
        mean_r = np.mean(self.lat.positions, axis=0)
        # distance from center
        delta_r = np.sqrt(np.sum((self.lat.positions - mean_r)**2, axis=1))
        # order sites by proximity to center
        order = np.argsort(delta_r)
        # fill half of sites closest to center with charge
        cutoff = self.lat.nsites // 2
        max_charge = 1
        self.lat.vals[order[0:cutoff]] = max_charge
        
    def dynamics(self, t):
        """Run the dynamics of the lattice. Apply gates serially at random locations.
        
        :param t: Positive integer. The number of gates applied is t*nsites.
        """
        for i in range(t*self.lat.nsites):
            rand_i = np.random.choice(range(self.lat.nsites))
            self.gate(self.lat, rand_i)

### Load profilers
You can install the profilers using `$ pip install line_profiler` and `$ pip install memory_profiler`.

In [ ]:
%load_ext line_profiler
%load_ext memory_profiler

### Initialize the demo

In [3]:
# extent of lattice
L = [25, 25]
# create demo
%timeit Demo(L)
d = Demo(L)

13.4 ms ± 1.29 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Put charge in the lattice

In [4]:
%timeit d.charge_blob()

164 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Run rounds of dynamics on the lattice to let the charge diffuse and plot

In [10]:
# timestep, apply dt * nsites gates
dt = 1
%timeit d.dynamics(dt)

302 ms ± 45.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
dt = 2
%timeit d.dynamics(dt)

616 ms ± 32.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
dt = 8
%timeit d.dynamics(dt)

2.91 s ± 264 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
dt = 16
%timeit d.dynamics(dt)

5.25 s ± 381 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
dt = 32
%timeit d.dynamics(dt)

11.1 s ± 276 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
